# Credit Risk Modeling with Lending Club Data
**Course Assignment — 2,000-word analytical report**

This notebook builds a credit risk scorecard using Lending Club personal loan data (2007–2018). The analysis covers three graded areas:

| Section | Topic | Weight |
|---------|-------|--------|
| 1 | Data Preparation | 30 % |
| 2 | Model Development | 60 % |
| 3 | Limitations | 10 % |


---
# 1. Data Preparation


## 1.1 Data Source & Sample Selection

The dataset is the **Lending Club loan book 2007–2018 Q4**, downloaded from Kaggle (`lending-club`). The raw accepted-loans file contains 678,151 observations across 151 columns. A **30 % stratified random sample** was drawn at read-time to stay within memory limits, yielding roughly 203,000 rows before status filtering.

Loans whose outcome had not yet been determined at extraction time were excluded:

- **Current** — still repaying, no outcome known  
- **In Grace Period** — overdue < 15 days, outcome uncertain  

After exclusion the **final modelling sample is 412,053 loans with a Bad Rate of 21.4 %**.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# check data
for f in os.listdir('/kaggle/input/lending-club'):
    size = os.path.getsize(f'/kaggle/input/lending-club/{f}')
    print(f"{f}: {size/1e9:.2f} GB")

import random

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Increase the accepted sampling rate since sufficient RAM is available
# Accepted loans are fewer in number, so a higher sample rate improves coverage

sample_rate_accepted = 0.3   # Sample 30% of accepted loans
sample_rate_rejected = 0.05  # Keep rejected loans at 5% (for reference only)

df_accepted = pd.read_csv(
    '/kaggle/input/lending-club/accepted_2007_to_2018Q4.csv.gz',
    skiprows=lambda x: x > 0 and random.random() > sample_rate_accepted,
    compression='gzip',
    low_memory=False
)

df_rejected = pd.read_csv(
    '/kaggle/input/lending-club/rejected_2007_to_2018Q4.csv.gz',
    skiprows=lambda x: x > 0 and random.random() > sample_rate_rejected,
    compression='gzip',
    low_memory=False
)

print(f"Accepted: {len(df_accepted)} rows")
print(f"Rejected: {len(df_rejected)} rows")

## 1.2 Target Variable Definition

A binary target is constructed from `loan_status`:

| Label | Value | Loan Statuses |
|-------|-------|---------------|
| **Bad** | 1 | Charged Off · Default · Late (31-120 days) · Late (16-30 days) · Does not meet credit policy: Charged Off |
| **Good** | 0 | Fully Paid · Does not meet credit policy: Fully Paid |

The "Does not meet credit policy" variants are mapped identically to their base status because the policy override is a platform flag unrelated to borrower performance.  
Rows with statuses other than the above are dropped (Current, In Grace Period) to ensure every observation has a definitive outcome.


In [ ]:
print("=== Accepted Loans — Columns ===")
print(df_accepted.columns.tolist())
print(f"\nTotal columns: {df_accepted.shape[1]}")

print("\n=== Rejected Loans — Columns ===")
print(df_rejected.columns.tolist())
print(f"\nTotal columns: {df_rejected.shape[1]}")

# Find columns common to both datasets
common_cols = set(df_accepted.columns) & set(df_rejected.columns)
print(f"\nShared columns: {common_cols}")

In [ ]:
# Case-insensitive column comparison
accepted_cols = set(c.lower().replace(' ', '_') for c in df_accepted.columns)
rejected_cols = set(c.lower().replace(' ', '_') for c in df_rejected.columns)
print(accepted_cols & rejected_cols)

In [ ]:
# Inspect loan_status distribution
print(df_accepted['loan_status'].value_counts())
print()
print(df_accepted['loan_status'].value_counts(normalize=True).round(3))

In [ ]:
# Exclude 'Current' and 'In Grace Period' — outcomes not yet determined
# Map remaining statuses to Good (0) and Bad (1)

bad_statuses = [
    'Charged Off',
    'Default',
    'Late (31-120 days)',
    'Late (16-30 days)',
    'Does not meet the credit policy. Status:Charged Off'
]

good_statuses = [
    'Fully Paid',
    'Does not meet the credit policy. Status:Fully Paid'
]

# Filter rows and define binary target variable
df_model = df_accepted[df_accepted['loan_status'].isin(bad_statuses + good_statuses)].copy()
df_model['target'] = df_model['loan_status'].isin(bad_statuses).astype(int)

# Inspect results
print(f"Modelling sample size: {len(df_model)}")
print(f"\nTarget distribution:")
print(df_model['target'].value_counts())
print()
print(df_model['target'].value_counts(normalize=True).round(3))

## 1.3 Feature Engineering & Preprocessing

### Data Leakage Removal

The accepted-loan file contains 151 columns, but **125+ of them are only populated after loan origination** (e.g., `total_pymnt`, `recoveries`, `last_pymnt_amnt`, `last_fico_range_*`). Including any of these would constitute data leakage — the model would be "predicting" outcomes using information that is generated by those very outcomes. All post-origination columns are excluded; only **application-time features** are retained.

### Missing Value Handling — Three Strategies

| Strategy | Variables | Rationale |
|----------|-----------|-----------|
| Fill with **0** | `mort_acc`, `pub_rec_bankruptcies` | Missing almost certainly means "none on record" — a zero is the meaningful value |
| Create **"Unknown" category** | `emp_length` | Missingness may carry predictive signal (e.g., self-employed, unemployed); masking it with a median would destroy that signal |
| Fill with **median** | `annual_inc`, `dti`, `delinq_2yrs`, `inq_last_6mths`, `open_acc`, `pub_rec`, `total_acc`, `revol_util` | Fewer than 120 rows affected; median is robust to outliers |

One row missing `zip_code` is dropped outright (negligible loss).

### Categorical Variable Consolidation

`zip_code` (840+ unique values) and `sub_grade` (35 values, highly collinear with `grade`) are dropped.  
`home_ownership` categories **ANY** and **NONE** are merged into **OTHER** because each represents < 0.1 % of observations — too sparse for stable WoE estimation.

### High-Correlation Variable Removal

Three pairs with Pearson |r| > 0.80 were identified:

| Pair | Correlation | Decision |
|------|-------------|----------|
| `fico_range_low` / `fico_range_high` | ≈ 1.00 | Drop `fico_range_high` — identical information |
| `loan_amnt` / `installment` | > 0.95 | Drop `installment` — `loan_amnt` is the primary underwriting variable |
| `grade_WoE` / `int_rate` | > 0.90 | Drop `int_rate` — `grade_WoE` is already linearity-transformed and more model-friendly |

### WoE Encoding & IV Filtering

Weight of Evidence (WoE) encoding converts categorical variables into continuous risk scores, making them directly usable in logistic regression without creating dummy-variable multicollinearity.

$$WoE_i = \ln\left(\frac{\text{Good}_i / \text{Total Good}}{\text{Bad}_i / \text{Total Bad}}\right)$$

$$IV = \sum_i (\%\text{Good}_i - \%\text{Bad}_i) \times WoE_i$$

After WoE encoding, `purpose_WoE` and `addr_state_WoE` were **excluded** because their IV values fell below 0.02 (no predictive power threshold). The remaining encoded variables — `grade_WoE`, `home_ownership_WoE`, `verification_status_WoE` — were retained.

### Engineered Feature: `loan_income_ratio`

A key insight from exploratory analysis: **current debt burden relative to income is more predictive than loan size or income in isolation**. The engineered feature is:

$$\text{loan\_income\_ratio} = \frac{\text{loan\_amnt}}{\text{annual\_inc}}$$

| Feature | IV | Predictive Strength |
|---------|-----|---------------------|
| `grade_WoE` | 0.449 | Very Strong |
| **`loan_income_ratio`** | **0.120** | **Medium** |
| `loan_amnt` | 0.039 | Weak |
| `annual_inc` | 0.024 | Weak |

The ratio's IV of 0.12 is **three times** that of its component features, confirming that the relative affordability metric captures risk dimensions neither feature captures alone. Values are winsorized at the 99th percentile to limit the influence of extreme outliers.

### Final Feature Set — 20 Variables

After all preprocessing, **20 features** enter the models:

`loan_amnt`, `term`, `emp_length`, `annual_inc`, `dti`, `delinq_2yrs`, `fico_range_low`, `inq_last_6mths`, `open_acc`, `pub_rec`, `revol_bal`, `revol_util`, `total_acc`, `credit_history_months`, `mort_acc`, `pub_rec_bankruptcies`, `grade_WoE`, `home_ownership_WoE`, `verification_status_WoE`, `loan_income_ratio`


In [ ]:
# Exclude post-origination variables (generated after loan issuance — data leakage)
leakage_cols = [
    'loan_status',           # Target variable itself
    'out_prncp',             # Remaining principal (only available after repayment)
    'out_prncp_inv',
    'total_pymnt',           # Total payment amount
    'total_pymnt_inv',
    'total_rec_prncp',       # Principal recovered
    'total_rec_int',         # Interest recovered
    'total_rec_late_fee',    # Late fees collected
    'recoveries',            # Recovery amount
    'collection_recovery_fee',
    'last_pymnt_d',          # Last payment date
    'last_pymnt_amnt',
    'next_pymnt_d',
    'last_credit_pull_d',
    'last_fico_range_high',  # Most recent FICO score (updated post-origination)
    'last_fico_range_low',
    'funded_amnt',           # Actual funded amount
    'funded_amnt_inv',
    'issue_d',               # Loan issuance date
    'url', 'desc', 'id', 'member_id'  # Non-informative ID columns
]

# Application-time features (available at underwriting — safe to use)
application_cols = [
    'loan_amnt',          # Requested loan amount
    'term',               # Loan term
    'int_rate',           # Interest rate
    'installment',        # Monthly installment
    'grade',              # Lending Club credit grade
    'sub_grade',          # Sub-grade (dropped later — collinear with grade)
    'emp_length',         # Employment length
    'home_ownership',     # Home ownership status
    'annual_inc',         # Annual income
    'verification_status',# Income verification status
    'purpose',            # Loan purpose
    'dti',                # Debt-to-income ratio
    'delinq_2yrs',        # Number of delinquencies in past 2 years
    'fico_range_low',     # FICO score at application time
    'fico_range_high',
    'inq_last_6mths',     # Credit inquiries in last 6 months
    'open_acc',           # Number of open credit accounts
    'pub_rec',            # Number of derogatory public records
    'revol_bal',          # Revolving credit balance
    'revol_util',         # Revolving credit utilization rate
    'total_acc',          # Total number of credit accounts
    'addr_state',         # US state
    'zip_code',           # ZIP code
    'earliest_cr_line',   # Earliest credit line date
    'mort_acc',           # Number of mortgage accounts
    'pub_rec_bankruptcies' # Number of public record bankruptcies
]

# Build clean modelling dataset
df_model_clean = df_model[application_cols + ['target']].copy()

print(f"Number of features: {len(application_cols)}")
print(f"\nMissing value summary:")
print(df_model_clean.isnull().sum()[df_model_clean.isnull().sum() > 0])

In [ ]:
df_clean = df_model_clean.copy()

# Convert earliest_cr_line to credit history length in months
import pandas as pd

reference_date = pd.Timestamp('2018-12-31')  # Reference date: end of dataset

df_clean['earliest_cr_line'] = pd.to_datetime(
    df_clean['earliest_cr_line'], format='%b-%Y'
)

df_clean['credit_history_months'] = (
    (reference_date.year - df_clean['earliest_cr_line'].dt.year) * 12 +
    (reference_date.month - df_clean['earliest_cr_line'].dt.month)
)

# Drop original date column now that it's been converted
df_clean = df_clean.drop(columns=['earliest_cr_line'])

# Fill remaining missing values (now numeric)
df_clean['credit_history_months'] = df_clean['credit_history_months'].fillna(
    df_clean['credit_history_months'].median()
)

print(df_clean['credit_history_months'].describe())

# 1. mort_acc and pub_rec_bankruptcies
# Missing almost certainly means 'none on record' — fill with 0
df_clean['mort_acc'] = df_clean['mort_acc'].fillna(0)
df_clean['pub_rec_bankruptcies'] = df_clean['pub_rec_bankruptcies'].fillna(0)

# 2. emp_length: high missingness — treat as its own 'Unknown' category
# Do not impute with median — missingness itself may carry predictive signal
df_clean['emp_length'] = df_clean['emp_length'].fillna('Unknown')

# 3. Numeric variables with very few missing values (<120 rows) — fill with median
small_missing = ['annual_inc', 'dti', 'delinq_2yrs', 
                 'inq_last_6mths', 'open_acc', 'pub_rec', 
                 'total_acc', 'revol_util']  # earliest_cr_line already converted

for col in small_missing:
    median_val = df_clean[col].median()
    df_clean[col] = df_clean[col].fillna(median_val)

# 4. zip_code has 1 missing row — drop it
df_clean = df_clean.dropna(subset=['zip_code'])

# Confirm no remaining missing values
print("Remaining missing values:")
remaining = df_clean.isnull().sum()
print(remaining[remaining > 0] if remaining[remaining > 0].any() else "No missing values!")

print(f"\nFinal sample size: {len(df_clean)}")
print(f"Bad Rate: {df_clean['target'].mean():.3f}")

In [ ]:
# Inspect distribution of all categorical variables
cat_cols = ['term', 'grade', 'sub_grade', 'emp_length', 
            'home_ownership', 'verification_status', 
            'purpose', 'addr_state', 'zip_code']

for col in cat_cols:
    n = df_clean[col].nunique()
    print(f"{col}: {n} unique values")
    if n <= 10:
        print(df_clean[col].value_counts())
    print()

In [ ]:
df_clean = df_clean.drop(columns=['zip_code', 'sub_grade'], errors='ignore')

# 3. Merge rare home_ownership categories into 'OTHER'
df_clean['home_ownership'] = df_clean['home_ownership'].replace({
    'ANY': 'OTHER',
    'NONE': 'OTHER'
})
print("home_ownership after merging rare categories:")
print(df_clean['home_ownership'].value_counts())

# 4. Convert term to numeric (only if still stored as string)
if df_clean['term'].dtype == object:
    df_clean['term'] = df_clean['term'].str.extract(r'(\d+)').astype(int)
print("\nterm after conversion to numeric:")
print(df_clean['term'].value_counts())

# 5. Convert emp_length to ordinal numeric (only if still string)
if df_clean['emp_length'].dtype == object:
    emp_map = {
        '< 1 year': 0, '1 year': 1, '2 years': 2, '3 years': 3,
        '4 years': 4, '5 years': 5, '6 years': 6, '7 years': 7,
        '8 years': 8, '9 years': 9, '10+ years': 10, 'Unknown': -1
    }
    df_clean['emp_length'] = df_clean['emp_length'].map(emp_map)
print("\nemp_length after ordinal encoding:")
print(df_clean['emp_length'].value_counts().sort_index())

In [ ]:
import numpy as np
import pandas as pd

def calculate_woe_iv(df, feature, target):
    """
    Calculate Weight of Evidence (WoE) and Information Value (IV).

    Returns:
    - woe_df  : per-category WoE and IV breakdown
    - iv      : overall Information Value
    - woe_map : dictionary for mapping categories to WoE values
    """
    # Build cross-tabulation
    ct = pd.crosstab(df[feature], df[target])
    ct.columns = ['Good', 'Bad']
    
    # Totals
    total_good = ct['Good'].sum()
    total_bad = ct['Bad'].sum()
    
    # Compute proportions (add small constant to avoid log(0))
    ct['Good_Pct'] = (ct['Good'] + 0.5) / total_good
    ct['Bad_Pct'] = (ct['Bad'] + 0.5) / total_bad
    
    # Compute WoE and IV
    ct['WoE'] = np.log(ct['Good_Pct'] / ct['Bad_Pct'])
    ct['IV_component'] = (ct['Good_Pct'] - ct['Bad_Pct']) * ct['WoE']
    
    # Compute Bad Rate per category
    ct['Bad_Rate'] = ct['Bad'] / (ct['Good'] + ct['Bad'])
    ct['Count'] = ct['Good'] + ct['Bad']
    
    iv = ct['IV_component'].sum()
    woe_map = ct['WoE'].to_dict()
    
    return ct[['Count', 'Bad_Rate', 'Good_Pct', 'Bad_Pct', 'WoE', 'IV_component']], iv, woe_map


# Calculate WoE and IV for all categorical variables
cat_cols = ['grade', 'home_ownership', 'verification_status', 'purpose', 'addr_state']

iv_summary = {}
woe_maps = {}

for col in cat_cols:
    woe_df, iv, woe_map = calculate_woe_iv(df_clean, col, 'target')
    iv_summary[col] = iv
    woe_maps[col] = woe_map
    
    print(f"\n{'='*50}")
    print(f"Variable: {col}  |  IV = {iv:.4f}")
    print(woe_df.sort_values('Bad_Rate', ascending=False).round(4))

# Summary of IV values
print(f"\n{'='*50}")
print("\nIV Summary (ranked by predictive power):")
iv_df = pd.DataFrame.from_dict(iv_summary, orient='index', columns=['IV'])
iv_df = iv_df.sort_values('IV', ascending=False)

def iv_strength(iv):
    if iv < 0.02: return 'No power'
    elif iv < 0.1: return 'Weak'
    elif iv < 0.3: return 'Medium'
    elif iv < 0.5: return 'Strong'
    else: return 'Very Strong'

iv_df['Strength'] = iv_df['IV'].apply(iv_strength)
print(iv_df)

In [ ]:
# Map WoE values back into the dataset
for col in cat_cols:
    df_clean[f'{col}_WoE'] = df_clean[col].map(woe_maps[col])
    
# Drop original categorical columns
df_clean = df_clean.drop(columns=cat_cols)

# Confirm final column list
print("\nFinal column list:")
print(df_clean.columns.tolist())
print(f"\nTotal columns (incl. target): {df_clean.shape[1]}")
print(f"Sample size: {df_clean.shape[0]}")

# Confirm no missing values
print(f"\nMissing values: {df_clean.isnull().sum().sum()}")

In [ ]:
# 1. Drop WoE-encoded variables with no predictive power (IV < 0.02)
df_clean = df_clean.drop(columns=['purpose_WoE', 'addr_state_WoE'])

# 2. Check correlation between grade_WoE and int_rate
print("Correlation between grade_WoE and int_rate:")
print(df_clean[['grade_WoE', 'int_rate']].corr())

# 3. Check correlation between fico_range_low and fico_range_high
print("\nCorrelation between fico_range_low and fico_range_high:")
print(df_clean[['fico_range_low', 'fico_range_high']].corr())

# 4. Full correlation matrix — identify highly correlated variable pairs
corr_matrix = df_clean.drop(columns='target').corr().abs()

# Identify pairs with Pearson |r| > 0.8
high_corr = []
cols = corr_matrix.columns
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        if corr_matrix.iloc[i, j] > 0.8:
            high_corr.append({
                'var1': cols[i],
                'var2': cols[j],
                'corr': corr_matrix.iloc[i, j]
            })

print("\nHighly correlated variable pairs (|r| > 0.8):")
print(pd.DataFrame(high_corr).sort_values('corr', ascending=False))

In [ ]:
# Remove highly correlated duplicate variables
df_clean = df_clean.drop(columns=[
    'fico_range_high',   # Identical to fico_range_low
    'installment',       # Highly correlated with loan_amnt
    'int_rate'           # Highly correlated with grade_WoE; grade_WoE is more model-friendly
])

# Confirm final column list
print("Final features:")
for col in df_clean.columns:
    print(f"  {col}")

print(f"\nTotal columns (incl. target): {df_clean.shape[1]}")
print(f"Number of features: {df_clean.shape[1] - 1}")

# Double-check: no remaining high-correlation pairs
corr_matrix = df_clean.drop(columns='target').corr().abs()
high_corr = []
cols = corr_matrix.columns
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        if corr_matrix.iloc[i, j] > 0.8:
            high_corr.append({
                'var1': cols[i],
                'var2': cols[j],
                'corr': round(corr_matrix.iloc[i, j], 4)
            })

if high_corr:
    print("\nStill-correlated variable pairs:")
    print(pd.DataFrame(high_corr).sort_values('corr', ascending=False))
else:
    print("\nNo remaining high-correlation pairs.")

## 1.4 Train / Test Split

The modelling sample is split **80 / 20** using `train_test_split` with `stratify=y` to preserve the class ratio in both partitions.

| Partition | Rows | Bad Rate |
|-----------|------|----------|
| Train | 329,642 | 21.4 % |
| Test | 82,411 | 21.4 % |

Stratification ensures no information leakage through class imbalance drift between splits.


In [ ]:
from sklearn.model_selection import train_test_split

# Separate features and target variable
X = df_clean.drop(columns='target')
y = df_clean['target']

# 80/20 split with stratification to preserve Bad Rate in both partitions
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)

print(f"Train set: {X_train.shape[0]} rows")
print(f"Test set: {X_test.shape[0]} rows")
print(f"\nTrain Bad Rate: {y_train.mean():.3f}")
print(f"Test Bad Rate: {y_test.mean():.3f}")

---
# 2. Model Development


## 2.1 Model 1 — Logistic Regression

### Methodology

Logistic Regression serves as the primary scorecard model and the interpretability benchmark. Two preprocessing choices are applied before fitting:

1. **StandardScaler** — logistic regression's gradient descent is sensitive to feature scale; standardisation ensures all 20 features contribute on equal footing.  
2. **`class_weight='balanced'`** — automatically reweights the minority class (Bad = 1, 21.4 %) so the optimiser does not simply predict "Good" for every observation to minimise loss.

The decision boundary is:

$$P(\text{Bad}) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 x_1 + \cdots + \beta_p x_p)}}$$

### Results

| Metric | Value |
|--------|-------|
| **AUC** | **0.7096** |
| Gini | 0.4192 |
| KS Statistic | 0.3037 |

### Feature Coefficient Interpretation

The top three predictors by absolute standardised coefficient are:

| Rank | Feature | Sign | Business Logic |
|------|---------|------|----------------|
| 1 | `grade_WoE` | + | Higher WoE = lower risk grade → lower default probability ✓ |
| 2 | `term` | + | 60-month loans carry higher default risk than 36-month ✓ |
| 3 | `dti` | + | Higher debt-to-income ratio → higher default risk ✓ |

All coefficient directions align with credit risk intuition, providing a sanity check on model validity.


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (roc_auc_score, roc_curve, 
                             classification_report, confusion_matrix)
import matplotlib.pyplot as plt
import numpy as np

# 1. Standardise features (logistic regression is scale-sensitive)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 2. Instantiate and train model
lr = LogisticRegression(
    max_iter=1000,
    random_state=42,
    class_weight='balanced'  # Re-weight minority class to address imbalance
)
lr.fit(X_train_scaled, y_train)

# 3. Generate predictions
y_pred_proba_lr = lr.predict_proba(X_test_scaled)[:, 1]
y_pred_lr = lr.predict(X_test_scaled)

# 4. Evaluation metrics
auc_lr = roc_auc_score(y_test, y_pred_proba_lr)
gini_lr = 2 * auc_lr - 1

print("="*50)
print("Logistic Regression Results")
print("="*50)
print(f"AUC:  {auc_lr:.4f}")
print(f"Gini: {gini_lr:.4f}")

# KS Statistic
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_lr)
ks_lr = max(tpr - fpr)
print(f"KS:   {ks_lr:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_lr))

# 5. ROC Curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'Logistic Regression (AUC = {auc_lr:.4f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Logistic Regression')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# 6. Feature importance via coefficients
coef_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': lr.coef_[0]
}).sort_values('Coefficient', key=abs, ascending=False)

print("\nFeature coefficients (sorted by magnitude):")
print(coef_df.round(4))

## 2.2 Model 2 — XGBoost

### Why XGBoost?

Logistic Regression assumes a **linear relationship** between features and the log-odds of default. In reality, credit risk exhibits non-linear thresholds (e.g., DTI risk accelerates steeply above 40 %) and **interaction effects** (e.g., high DTI matters more for subprime FICO borrowers). XGBoost addresses both through sequential gradient-boosted decision trees.

Key conceptual differences from Logistic Regression:

| Aspect | Logistic Regression | XGBoost |
|--------|--------------------|----|
| Decision boundary | Linear hyperplane | Ensemble of trees |
| Interactions | Manual (need to create terms) | Automatic |
| Interpretability | High (coefficients) | Moderate (SHAP / feature importance) |
| Robustness to outliers | Lower | Higher |
| Industry standard | Scorecard-grade | Challenger model |

`scale_pos_weight` is set to `len(negatives)/len(positives)` to mirror logistic regression's class weighting. Early stopping (`patience=20`) prevents overfitting on the test-set `eval_set`.

### Results

| Metric | Value |
|--------|-------|
| **AUC** | **0.7211** |
| Gini | 0.4422 |
| KS Statistic | 0.3219 |

XGBoost improves AUC by **+1.15 pp** over Logistic Regression.

### Feature Importance

`grade_WoE` dominates with an importance score of **0.509**, confirming that Lending Club's own credit grade is the single most powerful predictor even in a non-linear model. `loan_income_ratio` appears in the top five, validating the value of the engineered feature.


In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=len(y_train[y_train==0]) / len(y_train[y_train==1]),  # Balance class weights
    random_state=42,
    eval_metric='auc',
    early_stopping_rounds=20
)

xgb.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=50
)

# Evaluate on test set
y_pred_proba_xgb = xgb.predict_proba(X_test)[:, 1]
y_pred_xgb = xgb.predict(X_test)

auc_xgb = roc_auc_score(y_test, y_pred_proba_xgb)
gini_xgb = 2 * auc_xgb - 1
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_pred_proba_xgb)
ks_xgb = max(tpr_xgb - fpr_xgb)

print("="*50)
print("XGBoost Results")
print("="*50)
print(f"AUC:  {auc_xgb:.4f}")
print(f"Gini: {gini_xgb:.4f}")
print(f"KS:   {ks_xgb:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_xgb))

# ROC Curve comparison: LR vs XGBoost
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'Logistic Regression (AUC = {auc_lr:.4f})')
plt.plot(fpr_xgb, tpr_xgb, label=f'XGBoost (AUC = {auc_xgb:.4f})')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Comparison')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Feature importance
feat_imp = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': xgb.feature_importances_
}).sort_values('Importance', ascending=False)

print("\nXGBoost Feature Importance:")
print(feat_imp.round(4))

In [ ]:
feat_imp = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': xgb.feature_importances_
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(10, 8))
plt.barh(feat_imp['Feature'][::-1], feat_imp['Importance'][::-1], color='steelblue')
plt.xlabel('Importance Score')
plt.title('XGBoost Feature Importance')
plt.tight_layout()
plt.show()

## 2.3 Model Comparison — All Nine Models

To provide a rigorous benchmark, **nine models** were trained on the same 80 % training set and evaluated on the held-out 20 % test set.


In [ ]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()
nb.fit(X_train_scaled, y_train)

y_pred_proba_nb = nb.predict_proba(X_test_scaled)[:, 1]
fpr_nb, tpr_nb, _ = roc_curve(y_test, y_pred_proba_nb)

auc_nb = roc_auc_score(y_test, y_pred_proba_nb)
gini_nb = 2 * auc_nb - 1
ks_nb = max(tpr_nb - fpr_nb)

print(f"Naive Bayes - AUC: {auc_nb:.4f} | Gini: {gini_nb:.4f} | KS: {ks_nb:.4f}")

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=8,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1  # Use all available CPU cores
)
rf.fit(X_train, y_train)

y_pred_proba_rf = rf.predict_proba(X_test)[:, 1]
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_proba_rf)

auc_rf = roc_auc_score(y_test, y_pred_proba_rf)
gini_rf = 2 * auc_rf - 1
ks_rf = max(tpr_rf - fpr_rf)

print(f"Random Forest - AUC: {auc_rf:.4f} | Gini: {gini_rf:.4f} | KS: {ks_rf:.4f}")

In [ ]:
import lightgbm as lgb

lgbm = lgb.LGBMClassifier(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    scale_pos_weight=len(y_train[y_train==0]) / len(y_train[y_train==1]),
    random_state=42,
    n_jobs=-1,
    verbose=-1
)
lgbm.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    callbacks=[lgb.early_stopping(20, verbose=False)]
)

y_pred_proba_lgbm = lgbm.predict_proba(X_test)[:, 1]
fpr_lgbm, tpr_lgbm, _ = roc_curve(y_test, y_pred_proba_lgbm)

auc_lgbm = roc_auc_score(y_test, y_pred_proba_lgbm)
gini_lgbm = 2 * auc_lgbm - 1
ks_lgbm = max(tpr_lgbm - fpr_lgbm)

print(f"LightGBM - AUC: {auc_lgbm:.4f} | Gini: {gini_lgbm:.4f} | KS: {ks_lgbm:.4f}")

In [ ]:
from sklearn.neural_network import MLPClassifier

nn = MLPClassifier(
    hidden_layer_sizes=(64, 32),  # Two hidden layers: 64 and 32 neurons
    activation='relu',
    max_iter=100,
    random_state=42,
    early_stopping=True,
    validation_fraction=0.1,
    verbose=True
)
nn.fit(X_train_scaled, y_train)

y_pred_proba_nn = nn.predict_proba(X_test_scaled)[:, 1]
fpr_nn, tpr_nn, _ = roc_curve(y_test, y_pred_proba_nn)

auc_nn = roc_auc_score(y_test, y_pred_proba_nn)
gini_nn = 2 * auc_nn - 1
ks_nn = max(tpr_nn - fpr_nn)

print(f"Neural Network - AUC: {auc_nn:.4f} | Gini: {gini_nn:.4f} | KS: {ks_nn:.4f}")

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

# LDA — fast, fit directly
lda = LinearDiscriminantAnalysis()
lda.fit(X_train_scaled, y_train)

y_pred_proba_lda = lda.predict_proba(X_test_scaled)[:, 1]
fpr_lda, tpr_lda, _ = roc_curve(y_test, y_pred_proba_lda)

auc_lda = roc_auc_score(y_test, y_pred_proba_lda)
gini_lda = 2 * auc_lda - 1
ks_lda = max(tpr_lda - fpr_lda)

print(f"LDA - AUC: {auc_lda:.4f} | Gini: {gini_lda:.4f} | KS: {ks_lda:.4f}")

# LinearSVC — fast linear SVM implementation
# LinearSVC does not output probabilities natively — wrap with CalibratedClassifierCV
svm = CalibratedClassifierCV(
    LinearSVC(
        class_weight='balanced',
        max_iter=2000,
        random_state=42
    )
)
svm.fit(X_train_scaled, y_train)

y_pred_proba_svm = svm.predict_proba(X_test_scaled)[:, 1]
fpr_svm, tpr_svm, _ = roc_curve(y_test, y_pred_proba_svm)

auc_svm = roc_auc_score(y_test, y_pred_proba_svm)
gini_svm = 2 * auc_svm - 1
ks_svm = max(tpr_svm - fpr_svm)

print(f"SVM (Linear) - AUC: {auc_svm:.4f} | Gini: {gini_svm:.4f} | KS: {ks_svm:.4f}")

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train_scaled, y_train)

y_pred_proba_qda = qda.predict_proba(X_test_scaled)[:, 1]
fpr_qda, tpr_qda, _ = roc_curve(y_test, y_pred_proba_qda)

auc_qda = roc_auc_score(y_test, y_pred_proba_qda)
gini_qda = 2 * auc_qda - 1
ks_qda = max(tpr_qda - fpr_qda)

print(f"QDA - AUC: {auc_qda:.4f} | Gini: {gini_qda:.4f} | KS: {ks_qda:.4f}")

### Full Comparison Table


In [ ]:
plt.figure(figsize=(10, 7))

# Plot in descending AUC order
plt.plot(fpr_xgb, tpr_xgb, label=f'XGBoost (0.7202)', linewidth=2)
plt.plot(fpr_nn, tpr_nn, label=f'Neural Network (0.7158)', linewidth=2)
plt.plot(fpr_lgbm3, tpr_lgbm3, label=f'LightGBM (0.7105)', linewidth=2)
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (0.7103)', linewidth=2)
plt.plot(fpr, tpr, label=f'Logistic Regression (0.7101)', linewidth=2)
plt.plot(fpr_svm, tpr_svm, label=f'SVM Linear (0.7099)', linewidth=2)
plt.plot(fpr_lda, tpr_lda, label=f'LDA (0.7086)', linewidth=2)
plt.plot(fpr_nb, tpr_nb, label=f'Naive Bayes (0.6903)', linewidth=2)
plt.plot(fpr_qda, tpr_qda, label=f'QDA (0.6888)', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', label='Random', linewidth=1)

plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve - All Models Comparison', fontsize=14)
plt.legend(loc='lower right', fontsize=9)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Discussion — Three Performance Tiers

**Tier 1 — Complex non-linear models (XGBoost, Neural Network, LightGBM, Random Forest): AUC 0.710–0.721**  
These models outperform linear models by capturing non-linear risk thresholds and interaction effects. The gap is modest (~1–2 pp AUC), suggesting the linear WoE-encoded features already capture most predictable signal.

**Tier 2 — Linear model cluster (Logistic Regression, SVM Linear, LDA): AUC 0.708–0.710**  
All three produce nearly identical AUC scores. This convergence is expected: WoE encoding has already **linearised** the relationship between each feature and the log-odds of default. Given identical input transformations, any linear classifier will reach approximately the same decision boundary. The result validates the quality of the WoE preprocessing pipeline.

**Tier 3 — Strong-assumption models (Naive Bayes, QDA): AUC 0.688–0.690**  
Both models make parametric distributional assumptions that are violated by the data. Naive Bayes assumes feature independence (clearly false — grade, DTI, and income are correlated). QDA assumes multivariate normality within each class. The penalty for these violated assumptions is ~2 pp AUC.

### Impact of `loan_income_ratio` on Logistic Regression

Re-training Logistic Regression **with** `loan_income_ratio` yields:

| Version | AUC | Gini | KS |
|---------|-----|------|----|
| Without ratio | 0.7060 | 0.4120 | 0.2980 |
| **With ratio** | **0.7096** | **0.4192** | **0.3037** |

The +0.36 pp AUC gain from a single engineered feature illustrates that **feature engineering is more impactful for linear models** than adding model complexity. Non-linear models (XGBoost) can discover interaction effects internally; linear models require explicit construction of those interactions.


In [ ]:
# 1. Create new feature
df_clean['loan_income_ratio'] = df_clean['loan_amnt'] / df_clean['annual_inc']

# 2. Basic distribution
print("loan_income_ratio basic statistics:")
print(df_clean['loan_income_ratio'].describe().round(3))
print(f"\nExtreme values (>5): {(df_clean['loan_income_ratio'] > 5).sum()}")

# 3. Assess predictive power using WoE/IV (consistent with other features)
# Bin the ratio first
df_clean['ratio_bin'] = pd.qcut(
    df_clean['loan_income_ratio'].clip(upper=5),  # Winsorise extreme values
    q=10, duplicates='drop'
)

ratio_woe = df_clean.groupby('ratio_bin').agg(
    Good=('target', lambda x: (x==0).sum()),
    Bad=('target', lambda x: (x==1).sum())
).reset_index()

total_good = (df_clean['target']==0).sum()
total_bad = (df_clean['target']==1).sum()

ratio_woe['pct_good'] = ratio_woe['Good'] / total_good
ratio_woe['pct_bad'] = ratio_woe['Bad'] / total_bad
ratio_woe['WoE'] = np.log(ratio_woe['pct_good'] / ratio_woe['pct_bad'])
ratio_woe['IV'] = (ratio_woe['pct_good'] - ratio_woe['pct_bad']) * ratio_woe['WoE']
ratio_woe['Bad_Rate'] = ratio_woe['Bad'] / (ratio_woe['Good'] + ratio_woe['Bad'])

print(f"\nloan_income_ratio IV = {ratio_woe['IV'].sum():.4f}")
print("\nBin-level detail:")
print(ratio_woe[['ratio_bin', 'Good', 'Bad', 'Bad_Rate', 'WoE']].round(4))

# 4. Compare IV against component features
print("\n\nIV comparison across features:")
iv_compare = {
    'grade_WoE': 0.4492,
    'loan_amnt': None,   # To be computed
    'annual_inc': None,  # To be computed
    'loan_income_ratio': ratio_woe['IV'].sum()
}

# Compute IV for loan_amnt and annual_inc
for col in ['loan_amnt', 'annual_inc']:
    bins = pd.qcut(df_clean[col], q=10, duplicates='drop')
    ct = df_clean.groupby(bins).agg(
        Good=('target', lambda x: (x==0).sum()),
        Bad=('target', lambda x: (x==1).sum())
    )
    ct['pct_good'] = ct['Good'] / total_good
    ct['pct_bad'] = ct['Bad'] / total_bad
    ct['WoE'] = np.log(ct['pct_good'] / ct['pct_bad'])
    ct['IV'] = (ct['pct_good'] - ct['pct_bad']) * ct['WoE']
    iv_compare[col] = ct['IV'].sum()

for k, v in sorted(iv_compare.items(), key=lambda x: x[1], reverse=True):
    print(f"  {k:25s}: {v:.4f}")

# 5. Visualise Bad Rate by loan_income_ratio bins
plt.figure(figsize=(10, 5))
plt.bar(range(len(ratio_woe)), ratio_woe['Bad_Rate']*100, color='steelblue')
plt.xticks(range(len(ratio_woe)), 
           [str(b) for b in ratio_woe['ratio_bin']], 
           rotation=45, ha='right')
plt.xlabel('Loan/Income Ratio Bin')
plt.ylabel('Bad Rate (%)')
plt.title(f'Bad Rate by Loan/Income Ratio (IV = {ratio_woe["IV"].sum():.4f})')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
from xgboost import XGBClassifier

# 1. Create new feature
df_clean['loan_income_ratio'] = np.where(
    df_clean['annual_inc'] > 0,
    (df_clean['loan_amnt'] / df_clean['annual_inc']).clip(upper=5),
    np.nan
)
df_clean['loan_income_ratio'].fillna(df_clean['loan_income_ratio'].median(), inplace=True)
print(f"loan_income_ratio created, median: {df_clean['loan_income_ratio'].median():.3f}")

# Drop the temporary binning column
df_clean = df_clean.drop(columns=['ratio_bin'], errors='ignore')

# Verify all columns are numeric
print(df_clean.dtypes[df_clean.dtypes == object])  # Should be empty

# 2. Build updated feature set
X_new = df_clean.drop(columns='target')
y_new = df_clean['target']

X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(
    X_new, y_new, test_size=0.2, random_state=42, stratify=y_new
)

# 3. Logistic Regression with new feature
scaler_new = StandardScaler()
X_train_new_scaled = scaler_new.fit_transform(X_train_new)
X_test_new_scaled = scaler_new.transform(X_test_new)

lr_new = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')
lr_new.fit(X_train_new_scaled, y_train_new)

y_pred_proba_lr_new = lr_new.predict_proba(X_test_new_scaled)[:, 1]
auc_lr_new = roc_auc_score(y_test_new, y_pred_proba_lr_new)
gini_lr_new = 2 * auc_lr_new - 1
fpr_lr_new, tpr_lr_new, _ = roc_curve(y_test_new, y_pred_proba_lr_new)
ks_lr_new = max(tpr_lr_new - fpr_lr_new)

print(f"\nLR (new) - AUC: {auc_lr_new:.4f} | Gini: {gini_lr_new:.4f} | KS: {ks_lr_new:.4f}")

# 4. XGBoost with new feature
xgb_new = XGBClassifier(
    n_estimators=300, max_depth=5, learning_rate=0.05,
    subsample=0.8, colsample_bytree=0.8,
    scale_pos_weight=len(y_train_new[y_train_new==0]) / len(y_train_new[y_train_new==1]),
    random_state=42, eval_metric='auc', early_stopping_rounds=20
)
xgb_new.fit(X_train_new, y_train_new,
            eval_set=[(X_test_new, y_test_new)], verbose=False)

y_pred_proba_xgb_new = xgb_new.predict_proba(X_test_new)[:, 1]
auc_xgb_new = roc_auc_score(y_test_new, y_pred_proba_xgb_new)
gini_xgb_new = 2 * auc_xgb_new - 1
fpr_xgb_new, tpr_xgb_new, _ = roc_curve(y_test_new, y_pred_proba_xgb_new)
ks_xgb_new = max(tpr_xgb_new - fpr_xgb_new)

print(f"XGB (new) - AUC: {auc_xgb_new:.4f} | Gini: {gini_xgb_new:.4f} | KS: {ks_xgb_new:.4f}")

# 5. Before vs after comparison
print(f"\n=== Before vs After Adding loan_income_ratio ===")
print(f"{'Model':<20} {'AUC':>8} {'Gini':>8} {'KS':>8}")
print(f"{'LR (original)':<20} {auc_lr:>8.4f} {gini_lr:>8.4f} {ks_lr:>8.4f}")
print(f"{'LR (new)':<20} {auc_lr_new:>8.4f} {gini_lr_new:>8.4f} {ks_lr_new:>8.4f}")
print(f"{'XGBoost (original)':<20} {auc_xgb:>8.4f} {gini_xgb:>8.4f} {ks_xgb:>8.4f}")
print(f"{'XGBoost (new)':<20} {auc_xgb_new:>8.4f} {gini_xgb_new:>8.4f} {ks_xgb_new:>8.4f}")

## 2.4 Business Application

### Cut-off Selection Principle

A model score (predicted default probability) must be converted into an **accept / reject decision** via a threshold. This threshold is called the **cut-off**. The optimal cut-off depends on the business objective:

- **Maximise profit** — balance the revenue from good loans against the loss from bad loans  
- **Minimise bad rate** — tighter cut-off, lower approval rate  
- **Maximise volume** — looser cut-off, higher bad rate  

### Business Assumptions

| Parameter | Value | Rationale |
|-----------|-------|-----------|
| Average loan amount | \$15,311 | Mean of sample |
| Annual interest rate | 12 % | Approximate Lending Club average |
| Loan term | 3 years | Dominant product term |
| Loss Given Default (LGD) | 60 % | Industry standard for unsecured consumer lending |
| Revenue per good loan | \$loan × 12 % × 3 = \$5,512 | Net interest income |
| Loss per bad loan | \$loan × 60 % = \$9,187 | Principal lost after recovery |

### Strategy Curve

The strategy curve plots **Acceptance Rate (%)** against **Bad Rate among Accepted (%)**. As the cut-off is relaxed (more loans approved), the bad rate rises. The curve quantifies the trade-off and allows management to set risk appetite.


In [ ]:
thresholds = np.arange(0.01, 1.0, 0.01)
profits = []
approval_rates = []
bad_rates_approved = []

for threshold in thresholds:
    # Approve loans where predicted default probability <= threshold
    approved_mask = y_pred_proba_lr <= threshold
    
    true_good = ((approved_mask) & (y_test == 0)).sum()
    true_bad = ((approved_mask) & (y_test == 1)).sum()
    
    total_profit = (true_good * PROFIT_GOOD) - (true_bad * LOSS_BAD)
    approval_rate = approved_mask.sum() / len(y_test)
    bad_rate = true_bad / approved_mask.sum() if approved_mask.sum() > 0 else 0
    
    profits.append(total_profit)
    approval_rates.append(approval_rate)
    bad_rates_approved.append(bad_rate)

best_idx = np.argmax(profits)
best_threshold = thresholds[best_idx]
best_profit = profits[best_idx]

print(f"Optimal Cut-off: {best_threshold:.2f}")
print(f"Approval Rate: {approval_rates[best_idx]:.1%}")
print(f"Bad Rate (Approved): {bad_rates_approved[best_idx]:.1%}")
print(f"Maximum Profit: ${best_profit:,.0f}")

# English titles to avoid font rendering warnings
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

axes[0].plot(thresholds, [p/1e6 for p in profits], 'b-', linewidth=2)
axes[0].axvline(x=best_threshold, color='red', linestyle='--',
                label=f'Optimal Cut-off = {best_threshold:.2f}')
axes[0].set_xlabel('Cut-off Threshold')
axes[0].set_ylabel('Total Profit (Millions $)')
axes[0].set_title('Profit vs Cut-off Threshold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(thresholds, [r*100 for r in approval_rates],
             'b-', linewidth=2, label='Approval Rate %')
axes[1].plot(thresholds, [r*100 for r in bad_rates_approved],
             'r-', linewidth=2, label='Bad Rate (Approved) %')
axes[1].axvline(x=best_threshold, color='green', linestyle='--',
                label=f'Optimal Cut-off = {best_threshold:.2f}')
axes[1].set_xlabel('Cut-off Threshold')
axes[1].set_ylabel('Rate (%)')
axes[1].set_title('Approval Rate & Bad Rate vs Cut-off')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.suptitle('Business Impact Analysis - Logistic Regression', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Strategy Curve
# X-axis: Acceptance Rate, Y-axis: Bad Rate among Approved
axes[0].plot([r*100 for r in approval_rates],
             [r*100 for r in bad_rates_approved],
             'b-o', linewidth=2, markersize=3)

# Mark optimal cut-off point
best_approval = approval_rates[best_idx] * 100
best_bad = bad_rates_approved[best_idx] * 100
axes[0].axhline(y=best_bad, color='red', linestyle='--', alpha=0.7)
axes[0].axvline(x=best_approval, color='red', linestyle='--', alpha=0.7,
                label=f'Optimal: {best_approval:.1f}% approval, {best_bad:.1f}% bad rate')

axes[0].set_xlabel('Acceptance Rate (%)')
axes[0].set_ylabel('Bad Rate among Approved (%)')
axes[0].set_title('Strategy Curve')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Profit Curve
axes[1].plot(thresholds, [p/1e6 for p in profits], 'b-', linewidth=2)
axes[1].axvline(x=best_threshold, color='red', linestyle='--',
                label=f'Optimal Cut-off = {best_threshold:.2f}\nProfit = ${best_profit/1e6:.1f}M')
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[1].fill_between(thresholds, [p/1e6 for p in profits], 0,
                     where=[p > 0 for p in profits],
                     alpha=0.2, color='green', label='Profitable region')
axes[1].fill_between(thresholds, [p/1e6 for p in profits], 0,
                     where=[p <= 0 for p in profits],
                     alpha=0.2, color='red', label='Loss region')

axes[1].set_xlabel('Cut-off Threshold')
axes[1].set_ylabel('Total Profit (Millions $)')
axes[1].set_title('Profit vs Cut-off Threshold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.suptitle('Business Impact Analysis - Logistic Regression', fontsize=14)
plt.tight_layout()
plt.show()

### Profit Curve & Optimal Cut-off

The profit curve reveals a clear maximum:

| Parameter | Value |
|-----------|-------|
| **Optimal cut-off** | **0.69** |
| Approval rate | 88.5 % |
| Bad rate (approved) | ~18 % |
| **Maximum total profit** | **~\$195 M** |

The profit curve rises steeply as the cut-off increases from 0 (approve all → massive losses) to 0.69 (approve only low-risk borrowers). Above 0.69 the curve declines because too many profitable good borrowers are rejected.

A key insight: **the profit-maximising threshold is not the threshold that minimises bad rate**. A strategy focused solely on bad rate reduction would set a much tighter cut-off, sacrificing significant revenue by rejecting borderline-but-profitable borrowers.

### Run Book

The Run Book translates model scores (0–1000 scale, higher = lower risk) into operating decision bands, enabling front-line credit analysts to make consistent decisions without running the model in real time.


In [ ]:
# Build Run Book from LR predicted probabilities
# Predict on full dataset (train + test combined)
X_all_scaled = scaler_new.transform(X_new)
y_pred_proba_all = lr_new.predict_proba(X_all_scaled)[:, 1]

# Convert default probability to score (lower probability = higher score = lower risk)
# Linear transformation to simulate a scorecard scale (0–1000)
score_all = 1000 - (y_pred_proba_all * 1000).astype(int)

# Build Run Book DataFrame
runbook_df = pd.DataFrame({
    'score': score_all,
    'target': y_new.values
}).sort_values('score', ascending=False).reset_index(drop=True)

# Bin scores into predefined bands
bins = [0, 400, 500, 550, 600, 625, 650, 675, 700, 750, 1001]
labels = ['<400', '400-499', '500-549', '550-599', '600-624', 
          '625-649', '650-674', '675-699', '700-749', '750+']

runbook_df['score_band'] = pd.cut(runbook_df['score'], bins=bins, labels=labels, right=False)

# Compute statistics per band (cumulative from high score to low)
bands = runbook_df.groupby('score_band', observed=True).agg(
    Apps=('target', 'count'),
    Marginal_Bads=('target', 'sum')
).reset_index()

bands['Marginal_Bad_Rate'] = bands['Marginal_Bads'] / bands['Apps']
bands['GB_Odds'] = (bands['Apps'] - bands['Marginal_Bads']) / bands['Marginal_Bads']

# Cumulate from highest score (lowest risk) to lowest score (highest risk)
# Acc Rate = approving all applicants above this score band
total_apps = bands['Apps'].sum()
total_bads = bands['Marginal_Bads'].sum()

# Reverse order so highest score band appears first
bands = bands.iloc[::-1].reset_index(drop=True)

bands['Cumulative_Apps'] = bands['Apps'].cumsum()
bands['Cumulative_Bads'] = bands['Marginal_Bads'].cumsum()
bands['Acc_Rate'] = (bands['Cumulative_Apps'] / total_apps * 100).round(1)
bands['Bads_among_Accepts'] = bands['Cumulative_Bads']
bands['Bad_Rate_among_Accepts'] = (bands['Cumulative_Bads'] / bands['Cumulative_Apps'] * 100).round(2)
bands['Goods_among_Accepts'] = bands['Cumulative_Apps'] - bands['Cumulative_Bads']

# Format output columns
runbook_display = bands[[
    'score_band', 'Apps', 'Marginal_Bads', 'Marginal_Bad_Rate',
    'GB_Odds', 'Acc_Rate', 'Bads_among_Accepts', 
    'Bad_Rate_among_Accepts', 'Goods_among_Accepts'
]].copy()

runbook_display.columns = [
    'Score Band', '# Apps', 'Marginal # Bads', 'Marginal Bad Rate',
    'G/B Odds', 'Acc Rate %', '# Bads among Accepts',
    'Bad Rate among Accepts %', '# Goods among Accepts'
]

runbook_display['Marginal Bad Rate'] = (runbook_display['Marginal Bad Rate'] * 100).round(2)
runbook_display['G/B Odds'] = runbook_display['G/B Odds'].round(2)

print("Run Book — Logistic Regression Model")
print("="*100)
print(runbook_display.to_string(index=False))

# Mark the optimal cut-off position (threshold = 0.69)
optimal_score = int(1000 - 0.69 * 1000)
print(f"\nOptimal cut-off corresponds to score: {optimal_score}")
print(f"(Default probability > 0.69 → rejected; score < {optimal_score} → rejected)")

---
# 3. Limitations

## 3.1 Selection Bias *(most critical)*

The most fundamental limitation is that **all model training data comes exclusively from approved loans**. Lending Club only records repayment outcomes for borrowers who passed the initial underwriting screen. Rejected applicants are never observed.

This creates a classic **survivorship bias** problem: the model is trained on a non-random, pre-filtered population. When deployed, it will be applied to the full applicant pool — including borrower profiles that the historic policy would have rejected. The model has no information about how those profiles behave.

Concretely: the accepted-loan file has 151 columns; the rejected-loan file has only 9 columns with **zero overlapping variables**. There is no feasible way to bridge these datasets to correct for selection bias. Any performance metric reported in this notebook is therefore an **upper bound** on real-world model performance.

## 3.2 Temporal Instability

Analysis of Bad Rate by issuance year reveals a dramatic upward trend: **13 % in 2009 → 28 % in 2018**. The training data spans the 2008–2009 Global Financial Crisis, a period of extraordinary economic stress, as well as the subsequent recovery and expansion.

A model trained on this full period learns a mixture of crisis and non-crisis credit behaviour. Its predictions may be poorly calibrated for any specific economic environment — underestimating risk in downturns and overestimating risk during expansions. Walk-forward or time-stratified validation would be needed to quantify this instability.


In [ ]:
# Bad Rate trend by issuance year
df_model['issue_year'] = pd.to_datetime(df_model['issue_d']).dt.year
df_model.groupby('issue_year')['target'].mean().plot()

## 3.3 Geographic Heterogeneity

The choropleth map shows substantial variation in bad rates across US states: **Southern states reach 28 %** while **Pacific Northwest states fall as low as 15 %**. This 13 pp range suggests that local economic conditions (unemployment, housing market, industry mix) materially influence default risk.

`addr_state` was excluded from the final feature set because its IV was below the 0.02 threshold after WoE encoding — likely because state-level risk is already partially captured by correlated variables such as income and FICO score. However, macro-regional economic shocks (e.g., a regional recession) would not be captured by the current model, creating a systematic blind spot.


In [ ]:
# 1. Compute bad rate by state
state_stats = df_model.groupby('addr_state').agg(
    Total=('target', 'count'),
    Bad=('target', 'sum')
).reset_index()
state_stats['Bad_Rate'] = state_stats['Bad'] / state_stats['Total'] * 100

# Exclude states with fewer than 100 observations
state_stats = state_stats[state_stats['Total'] >= 100]

print("Top 10 states by Bad Rate:")
print(state_stats.nlargest(10, 'Bad_Rate')[['addr_state', 'Total', 'Bad_Rate']].round(2))
print("\nBottom 10 states by Bad Rate:")
print(state_stats.nsmallest(10, 'Bad_Rate')[['addr_state', 'Total', 'Bad_Rate']].round(2))

# 2. Choropleth map (plotly is pre-installed on Kaggle)
import plotly.express as px

fig = px.choropleth(
    state_stats,
    locations='addr_state',
    locationmode='USA-states',
    color='Bad_Rate',
    scope='usa',
    color_continuous_scale='RdYlGn_r',  # Red=high bad rate, Green=low bad rate
    title='Bad Rate by State - Lending Club 2007-2018',
    labels={'Bad_Rate': 'Bad Rate (%)'}
)
fig.update_layout(
    geo=dict(bgcolor='white'),
    title_font_size=16
)
fig.show()

# 3. State-level averages to help explain geographic variation
state_detail = df_model.groupby('addr_state').agg(
    Bad_Rate=('target', lambda x: x.mean()*100),
    Avg_FICO=('fico_range_low', 'mean'),
    Avg_DTI=('dti', 'mean'),
    Avg_Income=('annual_inc', 'mean'),
    Count=('target', 'count')
).round(2)

state_detail = state_detail[state_detail['Count'] >= 100]
state_detail = state_detail.sort_values('Bad_Rate', ascending=False)

print("\nDetailed statistics — top 10 highest bad rate states:")
print(state_detail.head(10).to_string())

## 3.4 Simplified Profit Assumptions

The profit analysis in Section 2.4 rests on several simplifying assumptions that limit its precision as an absolute dollar forecast:

- **No cost of funds** — deploying \$15K per loan requires capital that has an opportunity cost or borrowing cost, typically 3–5 % p.a. for a P2P platform.  
- **No operational costs** — origination, servicing, and collections costs are omitted.  
- **No time value of money** — all cash flows are treated as occurring at origination rather than being discounted over the 3-year loan term.  
- **Homogeneous LGD** — a flat 60 % LGD is applied to all defaults; in practice, LGD varies with collateral, recovery efforts, and macroeconomic conditions.  

Despite these simplifications, the **relative comparison** across cut-offs remains valid for identifying the profit-maximising threshold. Absolute profit figures should be treated as directional estimates only.


---
*End of Report*  
*Total word count target: ~2,000 words (Data Preparation ≈ 600 · Model Development ≈ 1,200 · Limitations ≈ 200)*
